# Gem stats analysis - symbolic regression

In [1]:
import os
import pickle
from itertools import product

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from pycgp import probabilistic_mutation, point_mutation, single_mutation
from pycgp.gems import MatchByActiveStrategy, MatchSMStrategy, MatchPMStrategy


    

In [29]:
class DataIterator():
    def __init__(self, folder):
        self.mutations = [
          #  (probabilistic_mutation, MatchSMStrategy),
            (point_mutation, MatchPMStrategy),
            (single_mutation, MatchSMStrategy),
            (single_mutation, MatchByActiveStrategy),
            (probabilistic_mutation, MatchByActiveStrategy)
        ]
        self.gems = [5, 10]
        
        self.cols = [10, 50, 100]
        
        data = []
        for m,s,g,c,d in self.__iterate_folder(folder):
            data.append([m, s, g, c, [x['gem_data'] for x in d], np.mean([x['gem_better_after'] for x in d]), np.mean([x['gem_worse_after'] for x in d]), np.mean([x['gem_same_after'] for x in d])])
        data = pd.DataFrame(data)
        data.columns = ['m', 's', 'g', 'c', 'gem_data', 'better_after', 'worse_after', 'safe_after']
        self.data = data      
    
    def __iterate_folder(self,folder):
        for index, ((mutation, strategy), gem, column) in enumerate(product(self.mutations, self.gems, self.cols)):
            file = os.path.join(folder,  f'{mutation.__name__}-{strategy.__name__}-gems{gem}-n_cols{column}.csv')
            with open (file, 'rb') as fp:
                data = pickle.load(fp)

                yield mutation.__name__, strategy.__name__, gem, column, data
    
    def iterate_gem_data(self, mutation, strategy, axis=False):
        gdatas = self.data[(self.data.m == mutation) & (self.data.s == strategy) & (self.data.g != 0)]

        if axis:
            _, axs = plt.subplots(2, len(gdatas)//2, figsize=(8,6))

        for i, (_, gdata) in enumerate(gdatas.iterrows()):
            pgdata = []
            #pdb.set_trace()
            for gem in [item for sublist in gdata.gem_data for item in sublist]:
                row = [gdata.g, gdata.c, gem.match_checks, gem.match_count, gem.n_uses, gem.value, gem.match_probability, gdata.better_after, gdata.worse_after, gdata.safe_after]
                pgdata.append(row)

            pgdata = pd.DataFrame(pgdata)
            pgdata.columns = ['gems', 'columns', 'match_checks', 'match_count', 'n_uses', 'value', 'match_probability', 'better_after', 'worse_after', 'same_after']
            pgdata['success_rate_counts'] = pgdata.n_uses / pgdata.match_count
            pgdata['success_rate_checks'] = pgdata.n_uses / pgdata.match_checks
            
            if axis:
                yield pgdata, axs[i//3][i%3]
            else:
                yield pgdata, None
    
    def stats(self):
        frames = []
        for m, s in self.mutations:
            for pgdata, _ in self.iterate_gem_data(m.__name__, s.__name__):
                pgdata['m'] = m.__name__
                pgdata['s'] = s.__name__
                frames.append(pgdata)
        data = pd.concat(frames)
        return data

import pdb
            

symreg = DataIterator('scripts/symbolic_basic/')
bincls = DataIterator('scripts/bin_class_out/')
santaf = DataIterator('scripts/santa_fe_out/')

def densities(di, m, s):
    frames = []
    for pgdata, ax in di.iterate_gem_data(m, s, True):
        frames.append(pgdata)
        sns.distplot(pgdata.n_uses, ax=ax)
    data = pd.concat(frames)
    print(f'Count of gems: {len(data)}')
    print(data.groupby('gems').median())


In [30]:
symreg.stats().groupby(['m', 's']).mean()

gems    columns  \
m                      s                                            
point_mutation         MatchPMStrategy        7.995846  55.981308   
probabilistic_mutation MatchByActiveStrategy  7.646771  53.884540   
single_mutation        MatchByActiveStrategy  7.826855  58.678445   
                       MatchSMStrategy        8.167239  54.879725   

                                              match_checks  match_count  \
m                      s                                                  
point_mutation         MatchPMStrategy        17033.717549   456.238837   
probabilistic_mutation MatchByActiveStrategy   1388.954990    32.749511   
single_mutation        MatchByActiveStrategy    911.676325    40.881979   
                       MatchSMStrategy         3542.781214     7.404353   

                                                 n_uses     value  \
m                      s                                            
point_mutation         MatchPMStrategy        18.500519  0.063370   
probabilistic_mutation MatchByActiveStrategy  25.086106  0.068130   
single_mutation        MatchByActiveStrategy  27.054417  0.046756   
                       MatchSMStrategy         7.153494  0.108973   

                                              match_probability  better_after  \
m                      s                                                        
point_mutation         MatchPMStrategy             7.258395e-24    127.972690   
probabilistic_mutation MatchByActiveStrategy       7.700268e-24     33.002104   
single_mutation        MatchByActiveStrategy       6.183401e-24     57.316537   
                       MatchSMStrategy             7.586752e-24     35.552978   

                                              worse_after  same_after  \
m                      s                                                
point_mutation         MatchPMStrategy          30.744496    5.605140   
probabilistic_mutation MatchByActiveStrategy    30.080479  156.474022   
single_mutation        MatchByActiveStrategy    21.474417  263.779576   
                       MatchSMStrategy          12.945361   11.049427   

                                              success_rate_counts  \
m                      s                                            
point_mutation         MatchPMStrategy                   0.714222   
probabilistic_mutation MatchByActiveStrategy             0.828059   
single_mutation        MatchByActiveStrategy             0.757424   
                       MatchSMStrategy                   0.981865   

                                              success_rate_checks  
m                      s                                           
point_mutation         MatchPMStrategy                   0.015125  
probabilistic_mutation MatchByActiveStrategy             0.074214  
single_mutation        MatchByActiveStrategy             0.115744  
                       MatchSMStrategy                   0.016860

# Overall stats

## Symbolic regression

In [31]:
sr = symreg.stats().groupby(['m','s','gems']).agg(['mean', 'count'])
sr.head()

columns        \
                                                        mean count   
m                      s                     gems                    
point_mutation         MatchPMStrategy       5     51.683938   386   
                                             10    58.856153   577   
probabilistic_mutation MatchByActiveStrategy 5     51.954262   481   
                                             10    55.600739   541   
single_mutation        MatchByActiveStrategy 5     57.674797   615   

                                                   match_checks        \
                                                           mean count   
m                      s                     gems                       
point_mutation         MatchPMStrategy       5     16459.059585   386   
                                             10    17418.150780   577   
probabilistic_mutation MatchByActiveStrategy 5      1363.081081   481   
                                             10     1411.959335   541   
single_mutation        MatchByActiveStrategy 5       839.001626   615   

                                                  match_count        \
                                                         mean count   
m                      s                     gems                     
point_mutation         MatchPMStrategy       5     354.279793   386   
                                             10    524.447140   577   
probabilistic_mutation MatchByActiveStrategy 5      30.665281   481   
                                             10     34.602588   541   
single_mutation        MatchByActiveStrategy 5      36.822764   615   

                                                      n_uses           value  \
                                                        mean count      mean   
m                      s                     gems                              
point_mutation         MatchPMStrategy       5     19.163212   386  0.066657   
                                             10    18.057192   577  0.061172   
probabilistic_mutation MatchByActiveStrategy 5     25.124740   481  0.069503   
                                             10    25.051756   541  0.066909   
single_mutation        MatchByActiveStrategy 5     27.055285   615  0.050043   

                                                         ...  better_after  \
                                                  count  ...          mean   
m                      s                     gems        ...                 
point_mutation         MatchPMStrategy       5      386  ...    111.564767   
                                             10     577  ...    138.949220   
probabilistic_mutation MatchByActiveStrategy 5      481  ...     32.448545   
                                             10     541  ...     33.494270   
single_mutation        MatchByActiveStrategy 5      615  ...     50.364878   

                                                        worse_after        \
                                                  count        mean count   
m                      s                     gems                           
point_mutation         MatchPMStrategy       5      386   25.155959   386   
                                             10     577   34.483102   577   
probabilistic_mutation MatchByActiveStrategy 5      481   28.679002   481   
                                             10     541   31.326525   541   
single_mutation        MatchByActiveStrategy 5      615   16.659593   615   

                                                   same_after        \
                                                         mean count   
m                      s                     gems                     
point_mutation         MatchPMStrategy       5       3.477591   386   
                                             10      7.028423   577   
probabilistic_mutation MatchByActiveStrategy 5     148.591892   481   
                             

## Binary classification

In [35]:
bc = bincls.stats().groupby(['m','s','gems']).agg(['mean', 'count'])

## Santa Fe

In [36]:
sf = santaf.stats().groupby(['m','s','gems']).agg(['mean', 'count'])

In [39]:
stats = pd.DataFrame()
for prep, d in zip(['SYM', 'BIN', 'SAN'], [sr, bc, sf]):
    stats[f'{prep}-count']   = d['success_rate_checks']['count'].values/60
  #  stats[f'{prep}-sucrate'] = d['success_rate_checks']['mean'].values
    stats[f'{prep}-better']  = d['better_after']['mean'].values
    stats[f'{prep}-worse']   = d['worse_after']['mean'].values
    stats[f'{prep}-same']   = d['same_after']['mean'].values
stats.index = sr.index
#stats.groupby(['m','s']).mean()
stats

SYM-count  SYM-better  \
m                      s                     gems                          
point_mutation         MatchPMStrategy       5      6.433333  111.564767   
                                             10     9.616667  138.949220   
probabilistic_mutation MatchByActiveStrategy 5      8.016667   32.448545   
                                             10     8.966667   33.494270   
single_mutation        MatchByActiveStrategy 5     10.250000   50.364878   
                                             10    13.316667   62.660625   
                       MatchSMStrategy       5      5.333333   29.180313   
                                             10     9.216667   39.240597   

                                                   SYM-worse    SYM-same  \
m                      s                     gems                          
point_mutation         MatchPMStrategy       5     25.155959    3.477591   
                                             10    34.483102    7.028423   
probabilistic_mutation MatchByActiveStrategy 5     28.679002  148.591892   
                                             10    31.326525  163.481978   
single_mutation        MatchByActiveStrategy 5     16.659593  230.226341   
                                             10    25.175812  289.573625   
                       MatchSMStrategy       5      8.499375    7.469062   
                                             10    15.518083   13.121248   

                                                   BIN-count  BIN-better  \
m                      s                     gems                          
point_mutation         MatchPMStrategy       5      5.416667   70.653692   
                                             10     8.050000   80.647619   
probabilistic_mutation MatchByActiveStrategy 5      7.150000    7.878322   
                                             10     7.833333    8.329255   
single_mutation        MatchByActiveStrategy 5      9.600000   19.185330   
                                             10    10.916667   22.875496   
                       MatchSMStrategy       5      4.883333   11.880205   
                                             10     8.750000   17.849429   

                                                   BIN-worse    BIN-same  \
m                      s                     gems                          
point_mutation         MatchPMStrategy       5      6.966308    2.644769   
                                             10    10.168944    4.606522   
probabilistic_mutation MatchByActiveStrategy 5      7.280420  183.446154   
                                             10     8.446277  207.297340   
single_mutation        MatchByActiveStrategy 5      4.373351  266.899913   
                                             10     8.203969  298.210153   
                       MatchSMStrategy       5      1.664676    4.795904   
                                             10     2.053048    7.490000   

                                                   SAN-count  SAN-better  \
m                      s                     gems                          
point_mutation         MatchPMStrategy       5      5.783333   85.839337   
                                             10    11.300000  136.117526   
probabilistic_mutation MatchByActiveStrategy 5      6.583333    7.260633   
                                             10     9.633333    6.858997   
single_mutation        MatchByActiveStrategy 5      8.250000   60.115524   
                                             10    13.916667   91.820441   
                       MatchSMStrategy       5      5.516667   48.291390   
                                             10    10.650000   80.863417   

                                                   SAN-worse    SAN-same  
m                      s                     gems                         
point_mutation         MatchPMStrategy       5     10.336455   10.430692  
                     

In [40]:
for i, r in stats.iterrows():
    print('{} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\'.format(r.name[2], *r.values))

5 & 6.43 & 111.56 & 25.16 & 3.48 & 5.42 & 70.65 & 6.97 & 2.64 & 5.78 & 85.84 & 10.34 & 10.43 \\
10 & 9.62 & 138.95 & 34.48 & 7.03 & 8.05 & 80.65 & 10.17 & 4.61 & 11.30 & 136.12 & 24.82 & 14.78 \\
5 & 8.02 & 32.45 & 28.68 & 148.59 & 7.15 & 7.88 & 7.28 & 183.45 & 6.58 & 7.26 & 6.89 & 138.93 \\
10 & 8.97 & 33.49 & 31.33 & 163.48 & 7.83 & 8.33 & 8.45 & 207.30 & 9.63 & 6.86 & 9.15 & 143.98 \\
5 & 10.25 & 50.36 & 16.66 & 230.23 & 9.60 & 19.19 & 4.37 & 266.90 & 8.25 & 60.12 & 12.42 & 150.58 \\
10 & 13.32 & 62.66 & 25.18 & 289.57 & 10.92 & 22.88 & 8.20 & 298.21 & 13.92 & 91.82 & 24.88 & 190.18 \\
5 & 5.33 & 29.18 & 8.50 & 7.47 & 4.88 & 11.88 & 1.66 & 4.80 & 5.52 & 48.29 & 9.24 & 9.90 \\
10 & 9.22 & 39.24 & 15.52 & 13.12 & 8.75 & 17.85 & 2.05 & 7.49 & 10.65 & 80.86 & 15.37 & 15.30 \\


# Symbolic regression

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

muts = [
    (point_mutation, MatchPMStrategy, 'Point mutation\nMatch all'),
    (single_mutation, MatchSMStrategy, 'Single mutation\nMatch all'),
    (single_mutation, MatchByActiveStrategy, 'Single mutation\n Match active'),
    (probabilistic_mutation, MatchByActiveStrategy, 'Prob. mutation\n Match active')
]

 
fig, axs = plt.subplots(1, len(muts), figsize=(16, 3))

data = pd.concat([symreg.stats(), bincls.stats(), santaf.stats()])
#plt.style.use('seaborn')
#  for ax, (m, s, t) in zip(axs, muts):
#      d = data[(data.m == m.__name__) & (data.s ==s.__name__)]['n_uses'].fillna(0)
#      d = sc.fit_transform(d)
#      sns.distplot(d, ax=ax)
#      ax.set_title(t)
print(data[data.m == 'point_mutation'].shape)
data['success_rate_checks'].reshape(2797,4)


In [ ]:
data = pd.concat([symreg.stats(), bincls.stats(), santaf.stats()])
fig, axs = plt.subplots(1, len(muts), figsize=(16, 3))
for ax, (m, s) in zip(axs, muts):
        sns.distplot(data[(data.m == m.__name__) & (data.s ==s.__name__)]['n_uses'], ax=ax)
        ax.set_xlim(0,30)

In [ ]:
densities(symreg, 'single_mutation', 'MatchSMStrategy')

In [ ]:
densities(symreg, 'single_mutation', 'MatchByActiveStrategy')

In [ ]:
densities(symreg, 'probabilistic_mutation', 'MatchByActiveStrategy')

In [ ]:
densities(symreg, 'point_mutation', 'MatchPMStrategy')

# Binary classification

In [ ]:
densities(bincls, 'single_mutation', 'MatchSMStrategy')

In [ ]:
densities(bincls, 'single_mutation', 'MatchByActiveStrategy')

In [ ]:
densities(bincls, 'probabilistic_mutation', 'MatchByActiveStrategy')

In [ ]:
densities(symreg, 'point_mutation', 'MatchPMStrategy')